In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.now())
print(f"{current_date}")

2025-05-04 14:20:02.056841


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date =current_date.floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-05-04T14:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

In [7]:
fetch_data_to

Timestamp('2025-05-04 14:00:00')

In [8]:
fetch_data_from

Timestamp('2025-04-06 14:00:00')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_bike_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_bike_data(years=[historical_from_date.year], months=[historical_from_date.month])
    rides_from = rides_from[rides_from.started_at >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_bike_data(years=[historical_to_date.year], months=[historical_to_date.month])
        rides_to = rides_to[rides_to.started_at < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides_from = rides_from[
            (rides_from.started_at >= historical_from_date) &
            (rides_from.started_at < historical_to_date)
        ]
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['started_at'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['start_station_id', 'started_at'], inplace=True)

    return rides

In [11]:
from src.data_utils import fetch_raw_data

fetch_raw_data(year=2024, month = 4)

c:\Users\nolan\cda500\citi_bike\src\data_utils.py:40: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


Saved 3217063 rows to ..\data\raw\citi_rides_2024_04.parquet


'..\\data\\raw\\citi_rides_2024_04.parquet'

In [18]:
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Optional, Tuple, Union

import numpy as np
import pandas as pd
import pytz
import requests
from io import BytesIO
import zipfile

def fetch_raw_data(year: int, month: int) -> str:
    """
    Download, extract, and convert Citi Bike trip data to Parquet format.
    The file is saved in RAW_DATA_DIR and is usable by GitHub Actions,
    even if the directory is in .gitignore.
    """
    path = Path('..') / "data" / "test" / f"citi_rides_{year}_{month:02}.parquet"
    path.parent.mkdir(parents=True, exist_ok=True)

    # If file already exists, return it
    if path.exists():
        print(f"Using cached file: {path}")
        return str(path)

    # Determine correct URL based on year/month
    if (year == 2024 and month in range(1, 5)) or (year == 2025 and month == 3):
        url = f"https://s3.amazonaws.com/tripdata/{year}{month:02}-citibike-tripdata.csv.zip"
    elif (year >= 2024 and month in range(5, 13)) or (year == 2025 and month in (1, 2)):
        url = f"https://s3.amazonaws.com/tripdata/{year}{month:02}-citibike-tripdata.zip"
    elif year < 2024:
        url = f"https://s3.amazonaws.com/tripdata/{year}-citibike-tripdata.zip"
    else:
        raise ValueError(f"No known data format for {year}-{month:02}")

    # Download the zip
    response = requests.get(url)
    if response.status_code != 200:
        raise FileNotFoundError(f"Could not download file from {url}")

    if year < 2024:
        with zipfile.ZipFile(BytesIO(response.content)) as outer_zip:
            target_prefix = f"{year}{month:02}"
            inner_zip_name = next(
                (f for f in outer_zip.namelist() if f.endswith(".zip") and target_prefix in f),
                None
            )
            if not inner_zip_name:
                raise FileNotFoundError(f"No nested zip file found for {target_prefix} in archive.")
            with outer_zip.open(inner_zip_name) as nested:
                with zipfile.ZipFile(BytesIO(nested.read())) as inner_zip:
                    dfs = _read_csvs_from_zip(inner_zip)
    else:
        with zipfile.ZipFile(BytesIO(response.content)) as z:
            dfs = _read_csvs_from_zip(z)

    if not dfs:
        raise ValueError("No CSV data found in zip file.")

    # Combine and save as Parquet
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.to_parquet(path, index=False)
    print(f"Saved {len(combined_df)} rows to {path}")
    return str(path)


def _read_csvs_from_zip(zip_file) -> list[pd.DataFrame]:
    """Helper to extract and parse CSVs inside a zip."""
    csv_files = [f for f in zip_file.namelist() if f.lower().endswith('.csv') and 'macosx' not in f.lower()]
    dfs = []
    for file in csv_files:
        with zip_file.open(file) as f:
            df = pd.read_csv(f)
            for col in ['start_station_id', 'end_station_id']:
                if col in df.columns:
                    df[col] = df[col].astype(str)
            dfs.append(df)
    return dfs

In [19]:
fetch_raw_data(2024, 4)

C:\Users\nolan\AppData\Local\Temp\ipykernel_27124\2349269823.py:73: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


Saved 3217063 rows to ..\data\test\citi_rides_2024_04.parquet


'..\\data\\test\\citi_rides_2024_04.parquet'

In [14]:
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Optional, Tuple, Union

import numpy as np
import pandas as pd
import pytz
import requests
from io import BytesIO
import zipfile
from pathlib import Path

from src.data_utils import filter_bike_data

def fetch_raw_data(year: int, month: int) -> str:
    """
    Download, extract, and convert Citi Bike trip data to Parquet format.
    The file is saved in RAW_DATA_DIR and is usable by GitHub Actions,
    even if the directory is in .gitignore.
    """
    path = Path("..") / "data" / "test" / f"citi_rides_{year}_{month:02}.parquet"
    path.parent.mkdir(parents=True, exist_ok=True)

    # If file already exists, return it
    if path.exists():
        print(f"Using cached file: {path}")
        return str(path)

    # Determine correct URL based on year/month
    if (year == 2024 and month in range(1, 5)) or (year == 2025 and month == 3):
        url = f"https://s3.amazonaws.com/tripdata/{year}{month:02}-citibike-tripdata.csv.zip"
    elif (year >= 2024 and month in range(5, 13)) or (year == 2025 and month in (1, 2)):
        url = f"https://s3.amazonaws.com/tripdata/{year}{month:02}-citibike-tripdata.zip"
    elif year < 2024:
        url = f"https://s3.amazonaws.com/tripdata/{year}-citibike-tripdata.zip"
    else:
        raise ValueError(f"No known data format for {year}-{month:02}")

    # Download the zip
    response = requests.get(url)
    if response.status_code != 200:
        raise FileNotFoundError(f"Could not download file from {url}")

    if year < 2024:
        with zipfile.ZipFile(BytesIO(response.content)) as outer_zip:
            target_prefix = f"{year}{month:02}"
            inner_zip_name = next(
                (f for f in outer_zip.namelist() if f.endswith(".zip") and target_prefix in f),
                None
            )
            if not inner_zip_name:
                raise FileNotFoundError(f"No nested zip file found for {target_prefix} in archive.")
            with outer_zip.open(inner_zip_name) as nested:
                with zipfile.ZipFile(BytesIO(nested.read())) as inner_zip:
                    dfs = _read_csvs_from_zip(inner_zip)
    else:
        with zipfile.ZipFile(BytesIO(response.content)) as z:
            dfs = _read_csvs_from_zip(z)

    if not dfs:
        raise ValueError("No CSV data found in zip file.")

    # Combine and save as Parquet
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.to_parquet(path, index=False)
    print(f"Saved {len(combined_df)} rows to {path}")
    return path

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_bike_data(years=[historical_from_date.year], months=[historical_from_date.month])
    rides_from = rides_from[rides_from.started_at >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_bike_data(years=[historical_to_date.year], months=[historical_to_date.month])
        rides_to = rides_to[rides_to.started_at < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides_from = rides_from[
            (rides_from.started_at >= historical_from_date) &
            (rides_from.started_at < historical_to_date)
        ]
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['started_at'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['start_station_id', 'started_at'], inplace=True)

    return rides

def _read_csvs_from_zip(zip_file) -> list[pd.DataFrame]:
    """Helper to extract and parse CSVs inside a zip."""
    csv_files = [f for f in zip_file.namelist() if f.lower().endswith('.csv') and 'macosx' not in f.lower()]
    dfs = []
    for file in csv_files:
        with zip_file.open(file) as f:
            df = pd.read_csv(f)
            for col in ['start_station_id', 'end_station_id']:
                if col in df.columns:
                    df[col] = df[col].astype(str)
            dfs.append(df)
    return dfs

def load_and_process_bike_data2(
    years: Optional[List[int]], months: Optional[List[int]] = None
    ) -> pd.DataFrame:
    """
    Load and process CitiBike ride data for a specified year and list of months.

    Args:
        year (Optional[List[int]]): List of years to load.
        months (Optional[List[int]]): List of months to load. If None, loads all months (1-12).

    Returns:
        pd.DataFrame: Combined and processed ride data for the specified year and months.

    Raises:
        Exception: If no data could be loaded for the specified year and months.
    """

    # Use all months if none are specified
    if months is None:
        months = list(range(1, 13))

    # List to store DataFrames for each month
    yearly_rides = []
    for year in years:
        monthly_rides = []
        if year == 2025:
            months = list(range(1,4))

        for month in months:
            # Construct the file path
            file_path = Path("..")/ "data" / "test" / f"citi_rides_{year}_{month:02}.parquet"

            try:
                # Download the file if it doesn't exist
                print(f"Downloading data for {year}-{month:02}...")
                fetch_raw_data(year, month)
                print(f"Successfully downloaded data for {year}-{month:02}.")


                # Load the data
                print(f"Loading data for {year}-{month:02}...")
                rides = pd.read_parquet(file_path, engine="pyarrow")

                # Filter and process the data
                rides = filter_bike_data(rides, year, month)
                print(f"Successfully processed data for {year}-{month:02}.")

                # Append the processed DataFrame to the list
                monthly_rides.append(rides)

            except FileNotFoundError:
                print(f"File not found for {year}-{month:02}. Skipping...")
            except Exception as e:
                print(f"Error processing data for {year}-{month:02}: {str(e)}")
                continue

        # Combine all monthly data
        if not monthly_rides:
            raise Exception(
                f"No data could be loaded for the year {year} and specified months: {months}"
            )

        print("Combining all monthly data...")
        month_combined_rides = pd.concat(monthly_rides, ignore_index=True)
        print(f"Data loading and processing for {year} complete!")
        yearly_rides.append(month_combined_rides)

    print("Combining all data...")
    combined_rides = pd.concat(yearly_rides, ignore_index=True)
    print("Data loading and processing complete!")

    return combined_rides.sort_values(by="started_at", ascending=True)

In [29]:
rides_from = load_and_process_bike_data2(years=[2024], months=[4])

Using cached file: ..\data\test\citi_rides_2024_04.parquet
Successfully downloaded data for 2024-04.
Loading data for 2024-04...
Total records: 3,217,063
Valid records: 34,543
Records dropped: 3,182,520 (98.93%)
Successfully processed data for 2024-04.
Combining all monthly data...
Data loading and processing for 2024 complete!
Combining all data...
Data loading and processing complete!


In [17]:
rides_from.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34543 entries, 32351 to 26709
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   started_at        34543 non-null  datetime64[ns]
 1   start_station_id  34543 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 809.6 KB


In [30]:
rides_from

,started_at,start_station_id
32351,2024-04-01 00:42:07.791,6822.09
20010,2024-04-01 00:50:18.615,5905.14
16331,2024-04-01 00:52:50.698,5905.14
3795,2024-04-01 00:53:10.769,6822.09
16326,2024-04-01 00:53:12.615,5905.14
...,...,...
27971,2024-04-30 23:11:33.882,6822.09
20733,2024-04-30 23:31:43.481,6822.09
4751,2024-04-30 23:33:11.672,6822.09
6959,2024-04-30 23:46:09.248,6822.09


In [32]:
from_date = "2025-04-06 20:00:00-04:00"
from_date = datetime.fromisoformat(from_date)
historical_from_date = from_date - timedelta(weeks=52)

In [33]:
timestamp = pd.Timestamp('2025-04-04 16:00:00', tz='US/Eastern')

# Remove timezone (make it naive)
naive_timestamp = timestamp.tz_localize(None)

print(timestamp)
print(naive_timestamp)  # Output will be naive (no timezone)


2025-04-04 16:00:00-04:00
2025-04-04 16:00:00


In [34]:
historical_from_date

datetime.datetime(2024, 4, 7, 20, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))

In [35]:
rides_from["started_at"] = rides_from["started_at"].dt.tz_localize("US/Eastern")

In [36]:
rides_from

,started_at,start_station_id
32351,2024-04-01 00:42:07.791000-04:00,6822.09
20010,2024-04-01 00:50:18.615000-04:00,5905.14
16331,2024-04-01 00:52:50.698000-04:00,5905.14
3795,2024-04-01 00:53:10.769000-04:00,6822.09
16326,2024-04-01 00:53:12.615000-04:00,5905.14
...,...,...
27971,2024-04-30 23:11:33.882000-04:00,6822.09
20733,2024-04-30 23:31:43.481000-04:00,6822.09
4751,2024-04-30 23:33:11.672000-04:00,6822.09
6959,2024-04-30 23:46:09.248000-04:00,6822.09


In [37]:
rides_from.started_at >= historical_from_date

32351    False
20010    False
16331    False
3795     False
16326    False
         ...  
27971     True
20733     True
4751      True
6959      True
26709     True
Name: started_at, Length: 34543, dtype: bool

In [20]:
df = pd.read_parquet(Path('..') / "data" / "test" / "citi_rides_2024_04.parquet")

In [21]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F561526822C9D60B,electric_bike,2024-04-27 13:56:13.940,2024-04-27 14:05:23.629,FDR Drive & E 35 St,6230.04,E 10 St & 2 Ave,5746.02,40.743955,-73.971391,40.729708,-73.986598,member
1,359BAF91507F4998,electric_bike,2024-04-25 15:23:14.529,2024-04-25 15:27:52.895,Forsyth St & Grand St,5382.07,E 10 St & 2 Ave,5746.02,40.717741,-73.993388,40.729708,-73.986598,member
2,AAEE95A1C0106C97,electric_bike,2024-04-06 11:15:18.132,2024-04-06 11:22:10.081,E 20 St & 2 Ave,5971.08,Mott St & Prince St,5561.04,40.735790,-73.981693,40.723180,-73.994800,member
3,95B077C9C619D404,electric_bike,2024-04-06 16:19:25.749,2024-04-06 16:21:43.098,Eastern Pkwy & Washington Ave,3928.08,Eastern Pkwy & Franklin Ave (SW Corner),3919.12,40.671649,-73.963115,40.670529,-73.958222,member
4,1A33C864454C4692,electric_bike,2024-04-10 17:40:14.700,2024-04-10 17:48:11.571,W 27 St & 6 Ave,6215.07,E 25 St & 1 Ave,6004.07,40.745446,-73.990591,40.738177,-73.977387,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217058,BE0F887BE16DFDDA,electric_bike,2024-04-16 07:47:35.966,2024-04-16 07:54:57.079,Atlantic Ave & Furman St,4614.04,Clark St & Henry St,4789.03,40.691736,-74.000174,40.697601,-73.993446,member
3217059,850F33001EDD1AE4,classic_bike,2024-04-10 08:45:16.500,2024-04-10 08:51:55.652,E 63 St & 3 Ave,6830.02,E 44 St & Lexington Ave,6464.09,40.763954,-73.964600,40.752643,-73.974996,member
3217060,CEE1CDE84344E8FF,electric_bike,2024-04-09 17:44:45.398,2024-04-09 18:03:46.293,Atlantic Ave & Furman St,4614.04,E 5 St & Cooper Sq,5712.12,40.691729,-74.000147,40.727690,-73.990993,member
3217061,174E104B28274EE1,classic_bike,2024-04-03 14:42:57.301,2024-04-03 14:51:34.967,W 48 St & Rockefeller Plaza,6626.11,W 20 St & 5 Ave,6098.02,40.757769,-73.979294,40.739730,-73.991040,member


In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-04.
Loading data for 2024-04...
Total records: 3,217,063
Valid records: 34,543
Records dropped: 3,182,520 (98.93%)
Successfully processed data for 2024-04.
Combining all monthly data...
Data loading and processing for 2024 complete!
Combining all data...
Data loading and processing complete!
File already exists for 2024-05.
Loading data for 2024-05...
Total records: 4,230,360
Valid records: 40,809
Records dropped: 4,189,551 (99.04%)
Successfully processed data for 2024-05.
Combining all monthly data...
Data loading and processing for 2024 complete!
Combining all data...
Data loading and processing complete!


In [19]:
rides

,started_at,start_station_id
11199,2025-05-03 13:00:13,5905.14
30064,2025-05-03 13:01:08,5905.14
20807,2025-05-03 13:04:30,5905.14
28407,2025-05-03 13:05:16,5905.14
37137,2025-05-03 13:06:05,5905.14
...,...,...
8564,2025-05-04 12:43:18,6822.09
5508,2025-05-04 12:44:52,6822.09
3923,2025-05-04 12:48:10,6822.09
38565,2025-05-04 12:52:45,6822.09


In [20]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [21]:
ts_data

,start_hour,start_station_id,rides
0,2025-05-03 13:00:00,5905.140137,68
1,2025-05-03 14:00:00,5905.140137,45
2,2025-05-03 15:00:00,5905.140137,47
3,2025-05-03 16:00:00,5905.140137,51
4,2025-05-03 17:00:00,5905.140137,44
...,...,...,...
67,2025-05-04 08:00:00,6822.089844,12
68,2025-05-04 09:00:00,6822.089844,7
69,2025-05-04 10:00:00,6822.089844,8
70,2025-05-04 11:00:00,6822.089844,10


In [22]:
import pytz

est = pytz.timezone('US/Eastern')

ts_data["start_hour"] = ts_data["start_hour"].dt.tz_localize(est, ambiguous="NaT", nonexistent="NaT")
ts_data = ts_data.dropna(subset=["start_hour"])
ts_data

,start_hour,start_station_id,rides
0,2025-05-03 13:00:00-04:00,5905.140137,68
1,2025-05-03 14:00:00-04:00,5905.140137,45
2,2025-05-03 15:00:00-04:00,5905.140137,47
3,2025-05-03 16:00:00-04:00,5905.140137,51
4,2025-05-03 17:00:00-04:00,5905.140137,44
...,...,...,...
67,2025-05-04 08:00:00-04:00,6822.089844,12
68,2025-05-04 09:00:00-04:00,6822.089844,7
69,2025-05-04 10:00:00-04:00,6822.089844,8
70,2025-05-04 11:00:00-04:00,6822.089844,10


In [23]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype                     
---  ------            --------------  -----                     
 0   start_hour        72 non-null     datetime64[ns, US/Eastern]
 1   start_station_id  72 non-null     float32                   
 2   rides             72 non-null     int16                     
dtypes: datetime64[ns, US/Eastern](1), float32(1), int16(1)
memory usage: 1.1 KB


In [24]:
ts_data.tail()

,start_hour,start_station_id,rides
67,2025-05-04 08:00:00-04:00,6822.089844,12
68,2025-05-04 09:00:00-04:00,6822.089844,7
69,2025-05-04 10:00:00-04:00,6822.089844,8
70,2025-05-04 11:00:00-04:00,6822.089844,10
71,2025-05-04 12:00:00-04:00,6822.089844,15


In [25]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')

# Initialize connection to Hopsworks
project = hopsworks.login(
    api_key_value=api_key,
    project=project_name
)
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-04 13:54:21,880 INFO: Initializing external client
2025-05-04 13:54:21,880 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-04 13:54:22,915 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Successfully connected to Hopsworks project: nolanphi


In [26]:
feature_store = project.get_feature_store()

In [27]:
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

In [28]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 72/72 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: bike_time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/bike_time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('bike_time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)